In [1]:
import os
import requests
import pandas as pd
from tensorboard.backend.event_processing import event_accumulator

In [2]:
tags = ['val_mean_distance', 'val_p1_acc', 'val_p5_acc', 'val_p10_acc']
log_root_dir = '/mnt/data2T/junyuan/eye-tracking/logs'
output_dir = '/mnt/data2T/junyuan/eye-tracking/outputs/TensorboarOutputs'
os.makedirs(output_dir, exist_ok=True)

In [3]:
def find_event_files(root_dir):
    """递归查找所有TensorBoard事件文件"""
    event_files = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.startswith('events.out.tfevents'):
                event_files.append(os.path.join(root, file))
    return event_files

def extract_experiment_name(event_file):
    """从文件路径中提取实验名称"""
    parts = event_file.split(os.sep)
    # 假设实验名称在路径的倒数第三层和倒数第二层
    experiment_name = f"{parts[-3]}_{parts[-2]}"
    return experiment_name

def save_scalars_to_csv(event_file, tags, output_dir):
    """保存标量数据为CSV文件"""
    ea = event_accumulator.EventAccumulator(event_file)
    ea.Reload()

    experiment_name = extract_experiment_name(event_file)

    # 初始化一个空的DataFrame
    df = pd.DataFrame()

    for tag in tags:
        if tag in ea.Tags()['scalars']:
            scalars = ea.Scalars(tag)
            tag_df = pd.DataFrame(scalars)
            tag_df.rename(columns={'value': tag}, inplace=True)
            if df.empty:
                df = tag_df[['step', tag]]
            else:
                df = pd.merge(df, tag_df[['step', tag]], on='step', how='outer')

    if not df.empty:
        output_file = os.path.join(output_dir, f'{experiment_name}.csv')
        df.to_csv(output_file, index=False)
        print(f'Saved {output_file}')

# 查找所有事件文件
event_files = find_event_files(log_root_dir)

# 保存每个事件文件中的标量数据为CSV文件
for event_file in event_files:
    save_scalars_to_csv(event_file, tags, output_dir)

Saved /mnt/data2T/junyuan/eye-tracking/outputs/TensorboarOutputs/EPNet_FixedCount5000_TrigERAugFPN2d_AllAug_version_0.csv
Saved /mnt/data2T/junyuan/eye-tracking/outputs/TensorboarOutputs/RGBUNet_version_0.csv
Saved /mnt/data2T/junyuan/eye-tracking/outputs/TensorboarOutputs/EventUNet_version_0.csv
Saved /mnt/data2T/junyuan/eye-tracking/outputs/TensorboarOutputs/ElNet_FixedCount5000_version_0.csv
Saved /mnt/data2T/junyuan/eye-tracking/outputs/TensorboarOutputs/EPNet_FixedCount5000_TrigERAugFPN2d_NoAllAug_version_0.csv
